## CONNECTING TO GOOGLE CLOUD STORAGE FROM LOCAL SPARK
In the last notebook we created a local session for spark and used the data saved locally. This time we want to pull data from GCS. In order to do that, we need to first upload the parquet files we created before to GCS. 

gsutil cp -m -r pq/ gs://ny-taxi-data-for-spark/pq

#### GET THE DATA FROM GOOGLE CLOUD
you will need to download a connector so that spark will understand how to read this URL 
You need the .jar file - cloud storage connector for Hadoop3 

https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [ ]:
spark.stop()

In [2]:
credentials_location = '/home/lulu/.gcp-keys/dbt-ny-taxi-key.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

Create the Spark Context. We previously created this with a builder. We won't use builder here. We will first create a context and then a session. The following says that when you see the file system beginning with gs:// that you need to use this implementation com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem that comes from the jar file we downloaded and the credentials we provided. 

confirmed that the class was in the version of the jar file I downloaded 
jar tf gcs-connector-hadoop3-latest.jar | grep GoogleHadoopFS


In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/02/14 17:56:08 WARN Utils: Your hostname, pepper resolves to a loopback address: 127.0.1.1; using 192.168.178.64 instead (on interface wlp2s0)
24/02/14 17:56:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/14 17:56:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 17:56:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


We have a configuration and context. Now we need to create a session. 

In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
!gsutil ls gs://ny-taxi-data-for-spark/pq/green/

gs://ny-taxi-data-for-spark/pq/green/2020/
gs://ny-taxi-data-for-spark/pq/green/2021/


In [8]:
df_green = spark.read.parquet('gs://ny-taxi-data-for-spark/pq/green/*/*')
#df_green = spark.read.parquet('gs://mage-zoomcamp-lulu-eu/green/*')

I had auth issues and had to add storage object permissions in GCP. 

In [9]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-12 18:15:04|  2020-01-12 18:19:52|                 N|         1|          41|          41|              1|         0.78|        5.5|  0.0|    0.